## CHAPTER 4 기본 신경망 구현

### 4.1 인공신경망의 작동 원리

### 4.2 간단한 분류 모델 구현하기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

In [3]:
# 기타 = [1, 0, 0]
# 포유류 = [0, 1, 0]
# 조류 = [0, 0, 1]

In [4]:
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [5]:
# # [털, 날개] -> [기타, 포유류, 조류]
# [0, 0] -> [1, 0, 0] # 기타
# [1, 0] -> [0, 1, 0] # 포유류
# [1, 1] -> [0, 0, 1] # 조류
# [0, 0] -> [1, 0, 0] # 기타
# [0, 0] -> [1, 0, 0] # 기타
# [0, 1] -> [0, 0, 1] # 조류

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [7]:
W = tf.Variable(tf.random_uniform([2,3], -1., 1.))
b = tf.Variable(tf.zeros([3]))

In [8]:
L = tf.add(tf.matmul(X, W), b)
L = tf.nn.relu(L)

In [9]:
model = tf.nn.softmax(L)

In [10]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

In [11]:
# 기본적인 경사하강법으로 최적화합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 텐서플로의 세션을 초기화합니다.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 앞서 구성한 특징과 레이블 데이터를 이용해 학습을 100번 진행합니다.
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    # 학습 도중 10번에 한 번씩 손실값을 출력해봅니다.
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.1742435
20 1.1713026
30 1.1684293
40 1.1656209
50 1.1628749
60 1.1601888
70 1.1575601
80 1.1549869
90 1.1524667
100 1.1499974


In [12]:
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 1 0 0 1]
실제값: [0 1 2 0 0 2]


In [13]:
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 66.67


### 4.3 심층 신경망 구현하기

In [14]:
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

In [15]:
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

In [16]:
model = tf.add(tf.matmul(L1, W2), b2)

In [17]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 0.9487025
20 0.80097914
30 0.66760606
40 0.5369539
50 0.4138522
60 0.3118552
70 0.23365255
80 0.17482042
90 0.13126586
100 0.09916533
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
